마케팅 사진을 만드는 코드입니다.  
과정은 다음과 같습니다.
1. chatGPT: 아이디어(input)->__마케팅 그림 묘사문__, 키워드
2. Stable Diffusion: 키워드->__그림__
3. Output: 그림(images)+묘사문(sentence)

In [ ]:
!pip install --upgrade diffusers accelerate transformers openai xformers

In [ ]:
import openai
import torch
from diffusers import DiffusionPipeline
from PIL import Image
openai.api_key="" #본인의 openai api key를 입력해주세요!

In [ ]:
input='making vegan burger'

In [ ]:
def answer(prompt:str):
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", 
                "content": prompt}
                ]
        )
        result=completion.choices[0].message["content"]
        return result

class text_generation():
    def __init__(self,input:str):
        self.input=input

    def returns(self):
        sentence=answer(f"You're a marketer. You're good at advertising and introducing products and services.\
        My service idea is {self.input}.\
        I want to make a picture for advertising idea, but I don't know what scene is suitable for marketing.\
        You can imagine a picture for idea. \
        Please describe the scene of photo for advertising idea in 4 lines.")
        
        keyword=answer(f"I write the description for photo.\
        description is {sentence}.\
        Please extract and list 8 keywords for the scene of photo. \
        You should write keywords using ',' symbol only. Other symbols must not be used.")

        return sentence, keyword

In [ ]:
sentence,keyword=text_generation(input).returns()

In [ ]:
pos_default="masterpiece, raw photo, extremely detailed skin, extremely detailed face, 8k uhd, \
dslr, soft lighting,  film grain, Fujifilm XT3, instagram, realstic" 

neg_default="(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4),\
text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, duplicate, morbid, mutilated,\
extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, blurry, dehydrated, bad anatomy,\
bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms,\
extra legs, fused fingers, too many fingers, long neck, (nsfw:2), (sexual content:2)"

class image_generation():
    def __init__(self,input:str,pos=pos_default,neg=neg_default):
        self.input=input
        self.pos=pos
        self.neg=neg
        repo_id="SG161222/Realistic_Vision_V2.0"
        self.pipeline=DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16).to("cuda")
        
    def implement(self):
        pos, neg=self.pos, self.neg
        pipeline=self.pipeline
        pipeline.enable_xformers_memory_efficient_attention()
        images = pipeline(num_inference_steps=30, 
                          prompt=pos+','+self.input,
                          negative_prompt=neg,
                          num_images_per_prompt=8).images
        return images

In [ ]:
images=image_generation(keyword).implement()

In [ ]:
class output_process():
    def __init__(self,image,sentence):
        def image_grid(imgs, rows=2, cols=4):
            assert len(imgs) == rows*cols

            w, h = imgs[0].size
            grid = Image.new('RGB', size=(cols*w, rows*h))
            
            for i, img in enumerate(imgs):
                grid.paste(img, box=(i%cols*w, i//cols*h))
            return grid
        self.image=image
        self.grid=image_grid(image)
        self.sentence=sentence
        
    def returns(self):
        return self.grid, self.sentence

    def show(self):
        self.grid.show()
        print(self.sentence)

In [ ]:
grid,sentence=output_process(images,sentence).returns()

In [ ]:
output_process(images,sentence).show()